In [80]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import requests
from bs4 import BeautifulSoup


import sys
sys.path.append('/Users/sambong/p/lib/')
import mongodb as mg
import list_handler as lh

db명 = '경력'
db = MongoClient()[db명]
data_path = '/Users/sambong/p/career/data/'

In [4]:
def PEIX로부터_practica를_수집후_저장():
    r = requests.get('https://www.inf.upv.es/int/peix/alumnos/listado_ofertas.php')
    
    # 파싱-1
    soup = BeautifulSoup(r.text, 'html.parser')
    tbls_bs = soup.find_all("table", class_="tabla_base")
    tbls_str = str(tbls_bs)
    df_li = pd.read_html( tbls_str )
    dic_li = []
    for df in df_li:
        df_1 = df.loc[:,[0,1]]
        df_1 = df_1.rename(columns={0:'key', 1:'val'})
        df_2 = df.loc[:,[2,3]]
        df_2 = df_2.rename(columns={2:'key', 3:'val'})
        df_3 = pd.concat([df_1, df_2]).dropna(axis=0, how='all')
        df_3.key = df_3.key.apply(lambda x: str(x).replace(' ','_') )
        df_3.key = df_3.key.apply(lambda x: str(x).replace(':','') )
        df_3.index = df_3.key
        del(df_3['key'])
        dic_li.append( df_3.val.to_dict() )

    # 파싱-2
    df = pd.DataFrame(dic_li)
    df = df.fillna('_')
    df.Titulaciones_ = df.Titulaciones_.combine(df.Titulación_, lambda x1, x2: x1 if x1 != '_' else x2)
    del(df['Titulación_'])
    df = df.rename(columns={
        'Bolsa':'월급',
        'Código_Oferta':'오퍼코드',
        'Empresa':'기업명',
        'Fecha_Inicio':'근무시작날짜',
        'Fecha_Oferta':'오퍼날짜',
        'Horas_al_Dia':'일근무시간',
        'Localidad':'근무지역', 
        'Perfil_de_la_Oferta':'요구능력',
        'Posibilidad_PFC':'PFC_가능성',
        'Tareas':'업무', 
        'Titulaciones_':'요구학력'})

    df.월급 = df.월급.str.replace(' Euros','')
    df.월급 = df.월급.apply(lambda x: float(x))
    df.일근무시간 = df.일근무시간.apply(lambda x: float(x))

    df.근무시작날짜 = df.근무시작날짜.apply(lambda x: datetime.strptime(x, '%d-%m-%Y'))
    df.오퍼날짜 = df.오퍼날짜.apply(lambda x: datetime.strptime(x, '%d-%m-%Y'))
    
    # 저장, 중복제거, 백업
    tbl명 = 'PEIX'
    db[tbl명].insert_many(df.to_dict('records'))
    mg.테이블의_중복제거(db명, tbl명, where=None, subset=['오퍼코드'])
    mg.테이블의_백업csv_생성(db명, tbl명, data_path)

In [123]:
def UPV_2018_Foro_기업정보_수집후_저장():
    # 수집
    r = requests.get('http://foroempleo.webs.upv.es/empresas_json.php')
    #'http://foroempleo.webs.upv.es/webapp/empresas/empresas_listado.html'

    # 파싱
    df1 = pd.DataFrame(r.json()['empresas'])
    df1 = df1.rename(columns={
        'stand':'부스번호',
        'logo':'회사로고',
        'nombre':'회사명',
        'fe_empresa_id':'회사_id',
        'fe_areasinteres_id':'산업분야_id',
        'areasinteres':'산업분야명_열거',
    })
    df1.회사_id = df1.회사_id.apply(lambda x: int(x))
    df1.산업분야_id = df1.산업분야_id.apply(lh.콤마스트링을_분리해서_리스트로_변환)

    # 저장, 백업 
    tbl명 = 'UPV_2018_Foro_참가기업'
    db[tbl명].insert_many(df1.to_dict('records'))
    mg.테이블의_중복제거(db명, tbl명, where=None, subset=['회사명'])
    mg.테이블의_백업csv_생성(db명, tbl명, data_path)

    # 파싱
    df2 = pd.DataFrame(r.json()['areasinteres'])
    df2 = df2.rename(columns={
        'fe_areasinteres_id':'산업분야_id',
        'nombre':'산업분야명',
        'orden':'순번'
    })

    # 저장, 백업 
    tbl명 = 'UPV_2018_Foro_산업분야정보'
    db[tbl명].insert_many(df2.to_dict('records'))
    mg.테이블의_중복제거(db명, tbl명, where=None, subset=None)
    mg.테이블의_백업csv_생성(db명, tbl명, data_path)